#Bài tập thực hành - Lập trình máy học cho python - CS116.M11



MSSV: 19521299

Tên: Nguyễn Chí Cường

Ngày: 9/11/2021

Tuần 11

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score, ShuffleSplit, GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import warnings
warnings.filterwarnings("ignore")

In [5]:
df = pd.read_csv('Social_Network_Ads.csv')
df

,Age,EstimatedSalary,Purchased
0,19,19000,0
1,35,20000,0
2,26,43000,0
3,27,57000,0
4,19,76000,0
...,...,...,...
395,46,41000,1
396,51,23000,1
397,50,20000,1
398,36,33000,0


In [6]:
scaler = StandardScaler()
df[['Age', 'EstimatedSalary']] = scaler.fit_transform(df[['Age', 'EstimatedSalary']])

feature_, class_ = df[['Age', 'EstimatedSalary']], df['Purchased']

# Bước 1: Hyper-Parameter SVM🥇

In [41]:
C = [0.1, 1, 10, 100, 1000]
gamma = [1, 0.1, 0.01, 0.001, 0.0001]
kernels = ['poly', 'linear', 'rbf', 'sigmoid']

param_grid ={'C': C, 'gamma': gamma, 'kernel': kernels}

#Bước 2: Chia dữ liệu Train, Validation, Test = 6:2:2🥈

In [62]:
def split_data(feature_, class_):
  ft, x_test, cls, y_test = train_test_split(feature_, class_, test_size=0.2, random_state=42)
  x_train, x_val, y_train, y_val = train_test_split(ft, cls, test_size=0.25, random_state=42)

  return x_train, y_train, x_val, y_val, x_test, y_test

x_train, y_train, x_val, y_val, x_test, y_test = split_data(feature_, class_)

#Bước 3: Tình best Hyper Parameter 🥉

In [63]:
#Use GridSearchCV and don't use data validation
train_data = np.concatenate((x_train, x_val)), np.concatenate((y_train, y_val))
grid_search = GridSearchCV(SVC(), param_grid, refit = True, cv = 5)
grid_search.fit(train_data[0], train_data[1])

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['poly', 'linear', 'rbf', 'sigmoid']})

In [72]:
print(grid_search.best_params_, "is best parameter for this data.")
print(grid_search.best_score_, "is best accuracy")

{'C': 1000, 'gamma': 0.1, 'kernel': 'rbf'} is best parameter for this data.
0.90625 is best accuracy


In [65]:
#code tay
def GridSearchSVM(param_grid, train, val):

  def update_paramater(C, gamma, kernel):
    return {'C': C, 'gamma': gamma, 'kernel': kernel}

  accuracy = 0
  parameter = {}
  model = SVC()

  for C in param_grid['C']:
    for gamma in param_grid['gamma']:
      for kernel in param_grid['kernel']:
        
        grid = SVC(C=C, gamma=gamma, kernel=kernel)
        grid.fit(train[0], train[1])
        predict = grid.predict(val[0])
        acc = accuracy_score(val[1], predict)

        if acc > accuracy:
          accuracy = acc
          parameter = update_paramater(C, gamma, kernel)
          model = grid
      
  return model, parameter, accuracy

#Bước 4: Tính Accuracy cho tập test ❄

In [66]:
#model by gridsearch in sklearn
skl_model = grid_search.best_estimator_
skl_acc = accuracy_score(skl_model.predict(x_test), y_test)
print("Giá trị accuracy trên tập test:", skl_acc)

Giá trị accuracy trên tập test: 0.925


In [70]:
#model tự kiếm
my_model, my_para, acc = GridSearchSVM(param_grid, [x_train, y_train], [x_val, y_val])
my_acc = accuracy_score(my_model.predict(x_test), y_test)
print("Giá trị accuracy trên tập test:", my_acc)

Giá trị accuracy trên tập test: 0.925


#End